# Inroduction


"""
Dynamic Pricing for Urban Parking Lots
Capstone Project — Summer Analytics 2025

This notebook implements a real-time dynamic pricing system for 14 urban parking lots,
based on varying demand and competitor prices using the Pathway streaming engine.

📊 Real-Time Bokeh Visualizations:

📈 Model 1: Baseline pricing (Occupancy)

📉 Model 2: Demand-based pricing

🏷️ Model 3: Competitive pricing (within 0.5 km).

🧰 Tech Stack:
- [Pathway](https://pathway.com)
- [Bokeh](https://bokeh.org)
- [Panel](https://panel.holoviz.org/)
- [Pandas](https://pandas.pydata.org/), [NumPy](https://numpy.org/)


🗺️ Project Roadmap:
1. Load and preprocess live data for 14 parking lots.
2. Apply three pricing models:
   - Model 1: Linear price = Base + α * (occupancy/capacity)
   - Model 2: Demand score = weighted sum of inputs → price curve
   - Model 3: Adjust price based on cheaper nearby competitors (within 0.5 km)
3. Stream price outputs via Pathway and visualize each lot live using Bokeh.
4. Enable easy comparison across models via tabs or combined graphs.
        
Note:

- Designed for real-time simulation of incoming data (up to 18,000 rows).
- Optimized for smooth, low-latency plotting.
- Final version uses Pathway v0.24.1 and is compliant with the problem statement.

Author: [Vivan Khatri]

Date: July 6, 2025
"""


In [1]:
!pip install pathway bokeh  --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

# Step 1: Importing and Preprocessing the Data

In [3]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [4]:
df = pd.read_csv("dataset.csv")
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')

In [5]:
# Combine date and time columns into Timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort by timestamp
df = df.sort_values("Timestamp").reset_index(drop=True)

# Select all relevant columns (not just 3 like sample)
# You can add or remove based on your actual dataset
df = df[[
    "Timestamp", "SystemCodeNumber", "Capacity", "Occupancy", "Latitude",
    "Longitude", "QueueLength", "TrafficConditionNearby",
    "VehicleType", "IsSpecialDay"
]]


# Save for streaming
df.to_csv("processed_dataset.csv", index=False)


In [6]:
# Schema with Timestamp as string
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Capacity: int
    Occupancy: float
    Latitude: float
    Longitude: float
    QueueLength: float
    TrafficConditionNearby: str
    VehicleType: str
    IsSpecialDay: bool


In [7]:
# Save this again before starting streaming
df.to_csv("parking_stream.csv", index=False)


In [8]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [9]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


Model 1: Baseline Linear Model & its's bokeh plot

In [10]:
# Step 2: Define the pricing logic
def model_1_pricing(occupancy: float) -> float:
    base_price = 10
    slope = 0.1
    return base_price + slope * occupancy

# Step 3: Apply it row-wise
model_1_stream = data_with_time.select(
    t = data_with_time.t,
    SystemCodeNumber = data_with_time.SystemCodeNumber,
    price = pw.apply(model_1_pricing, data_with_time.Occupancy)
)


In [11]:
# Activate the Panel extension (only once at top of notebook)
pn.extension()

# Define the same Bokeh plotting function
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1: Price = 20 + 0.1 × Occupancy",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=6, color="red")
    return fig
# Step 1: Get all unique lot IDs from your dataset
lot_ids = df["SystemCodeNumber"].unique().tolist()

# Step 2: Build one tab per lot
tabs = []

for lot in lot_ids:
    # ✅ Filter first, before select
    lot_stream = model_1_stream.filter(pw.this.SystemCodeNumber == lot).select(
        pw.this.t,
        pw.this.price
    )

    # Plot
    viz1 = lot_stream.plot(
        lambda source: price_plotter(source),
        sorting_col="t"
    )

    tabs.append((f"Lot {lot}", viz1))

# Step 3: Create panel tabs
pn.Tabs(*tabs).servable()



Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

 **NOTE**:-THE PLOT WILL NOT RUN THE FIRST TIME YOU RUN PW.RUN() BUT IF WE GO BACK AND RE-RUN FROM THIS STEP

(Save this again before starting streaming

df.to_csv("parking_stream.csv", index=False))


 AND THEN RUN THE PW.RUN() AGAIN OR  JUST RE-RUN  THE CODE WITHOUT RESTARTTING THE RUN TIME ,IT WILL START PLOTTING AFTER A FEW SECS  !!
IDK WHY THIS HAPPENS BUT IT DOES WORK !!

In [12]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

Model 2: Demand-Based Price Function & it's bokeh plot

In [12]:
# --- Updated weights with lowercase support ---
vehicle_weights = {
    "2w": 0.4,
    "bike": 0.4,
    "4w": 0.6,
    "car": 0.6,
    "heavy": 0.8,
}

traffic_map = {
    "low": 0.5,
    "medium": 1.0,
    "high": 1.5,
}

# --- Updated Model 2 function ---
def model_2_pricing(occupancy, capacity, queue_length, traffic, is_special_day, vehicle_type):
    BasePrice = 10.0
    alpha = 12     # ↑ occupancy weight
    beta = 1.5     # ↓ queue weight
    gamma = 4      # ↓ traffic weight
    delta = 5      # keep
    epsilon = 12   # ↑ vehicle type weight
    lambda_ = 0.6  # ↑ scaling factor (from 0.5 to 0.6)


    try:
        traffic_val = traffic_map.get(str(traffic).strip().lower(), 0.5)
        vt_weight = vehicle_weights.get(str(vehicle_type).strip().lower(), 0.5)

        if capacity == 0:
            return BasePrice

        demand = (
            alpha * (occupancy / capacity)
            + beta * queue_length
            - gamma * traffic_val
            + (delta if is_special_day else 0)
            + epsilon * vt_weight
        )

        norm_demand = max(0.0, min(demand / 25, 1.0))  # smoother mapping
        price = BasePrice * (1 + lambda_ * norm_demand)
        return round(max(BasePrice * 0.5, min(price, BasePrice * 2)), 2)

    except Exception as e:
        print("❌ Error computing price for row:")
        print(f"→ Occ: {occupancy}, Cap: {capacity}, Queue: {queue_length}, Traffic: {traffic}, Day: {is_special_day}, VT: {vehicle_type}")
        print("Error:", e)
        return BasePrice


In [13]:
df["model_2_price"] = df.apply(lambda row: model_2_pricing(
    row["Occupancy"],
    row["Capacity"],
    row["QueueLength"],
    row["TrafficConditionNearby"],
    row["IsSpecialDay"],
    row["VehicleType"]
), axis=1)


In [14]:
print(df[["SystemCodeNumber", "model_2_price"]].head())
print(df["model_2_price"].describe())


  SystemCodeNumber  model_2_price
0      BHMBCCMKT01          11.91
1      BHMNCPHST01          11.96
2      BHMMBMMBX01          13.07
3      BHMNCPNST01          13.45
4         Shopping          12.60
count    18368.000000
mean        14.124968
std          1.111642
min         10.730000
25%         13.290000
50%         14.130000
75%         14.980000
max         16.000000
Name: model_2_price, dtype: float64


In [15]:
model_2_stream = data_with_time.with_columns(
    t = data_with_time.t,
    SystemCodeNumber = data_with_time.SystemCodeNumber,
    price = pw.apply(
        model_2_pricing,
        data_with_time.Occupancy,
        data_with_time.Capacity,
        data_with_time.QueueLength,
        data_with_time.TrafficConditionNearby,
        data_with_time.IsSpecialDay,
        data_with_time.VehicleType,
    )
)


In [16]:


def price_plotter_model_2(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 2: Demand-Based Dynamic Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.scatter("t", "price", source=source, size=6, color="orange")
    return fig

lot_ids = df["SystemCodeNumber"].unique().tolist()
tabs = []

for lot in lot_ids:
    lot_stream = model_2_stream.filter(pw.this.SystemCodeNumber == lot).select(
        pw.this.t,
        pw.this.price
    )
    viz2 = lot_stream.plot(
        lambda source: price_plotter_model_2(source),
        sorting_col="t"
    )
    tabs.append((f"Lot {lot}", viz2))


tabs_panel = pn.Tabs(*tabs)
tabs_panel.servable()


Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [34]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

AttributeError: 'Column' object has no attribute 'document'

Model 3 : Competitive Pricing Model & it's bokeh plot

In [17]:
# Keep one row per parking lot
lot_locations = df.drop_duplicates(subset=["SystemCodeNumber"])[["SystemCodeNumber", "Latitude", "Longitude"]].reset_index(drop=True)

# Check number of lots (should be 14)
print("Unique lots:", lot_locations.shape[0])

Unique lots: 14


In [18]:


# Convert to radians
codes = lot_locations["SystemCodeNumber"].values
lat = np.radians(lot_locations["Latitude"].values)
lon = np.radians(lot_locations["Longitude"].values)

# Build distance matrix
lat1, lat2 = np.meshgrid(lat, lat)
lon1, lon2 = np.meshgrid(lon, lon)

dlat = lat2 - lat1
dlon = lon2 - lon1
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
c = 2 * np.arcsin(np.sqrt(a))
dist_matrix = 6371 * c  # in kilometers

# Build competitor map (only if ≤ 0.5 km apart)
competitor_map = {code: [] for code in codes}
n = len(codes)

for i in range(n):
    for j in range(i + 1, n):
        if dist_matrix[i, j] <= 0.5:
            competitor_map[codes[i]].append(codes[j])
            competitor_map[codes[j]].append(codes[i])

# Show sample result
for k in list(competitor_map.keys())[:5]:
    print(f"{k}: {competitor_map[k]}")





BHMBCCMKT01: ['BHMBCCTHL01']
BHMNCPHST01: ['BHMNCPNST01']
BHMMBMMBX01: []
BHMNCPNST01: ['BHMNCPHST01']
Shopping: []


In [19]:
def model_3_price(row, df_lookup):
    lot = row["SystemCodeNumber"]
    t = row["Timestamp"]
    base_price = row["model_2_price"]

    competitors = competitor_map.get(lot, [])
    if not competitors:
        return base_price

    # Get competitor prices at the same timestamp
    prices = df_lookup.get((t, tuple(competitors)), [])
    if not prices:
        return base_price

    avg_price = sum(prices) / len(prices)
    return round(max(5, min(20, 0.5 * base_price + 0.5 * avg_price)), 2)


In [20]:
from collections import defaultdict

lookup = defaultdict(list)
for _, row in df.iterrows():
    key = (row["Timestamp"], row["SystemCodeNumber"])
    lookup[key].append(row["model_2_price"])

# Convert to new key: (t, tuple of competitors)
df_lookup = defaultdict(list)
for (ts, lot), prices in lookup.items():
    df_lookup[(ts, (lot,))] = prices


In [21]:
df["model_3_price"] = df.apply(lambda row: model_3_price(row, df_lookup), axis=1)


In [22]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

tabs = []

for lot in df["SystemCodeNumber"].unique():
    dlot = df[df["SystemCodeNumber"] == lot].sort_values("Timestamp")
    source = ColumnDataSource(dlot)

    fig = figure(
        height=400,
        width=800,
        title=f"Model 3 Price for {lot}",
        x_axis_type="datetime"
    )
    fig.line("Timestamp", "model_3_price", source=source, color="purple", line_width=2)
    fig.scatter("Timestamp", "model_3_price", source=source, color="orange", size=4)

    # ✅ Use Panel Tabs (not Bokeh Panel) for full compatibility
    tabs.append((str(lot), fig))

pn.Tabs(*tabs).servable()


Tabs
    [0] Bokeh(figure, name='BHMBCCMKT01')
    [1] Bokeh(figure, name='BHMNCPHST01')
    [2] Bokeh(figure, name='BHMMBMMBX01')
    [3] Bokeh(figure, name='BHMNCPNST01')
    [4] Bokeh(figure, name='Shopping')
    [5] Bokeh(figure, name='BHMEURBRD01')
    [6] Bokeh(figure, name='Broad Street')
    [7] Bokeh(figure, name='Others-CCCPS8')
    [8] Bokeh(figure, name='Others-CCCPS105a')
    [9] Bokeh(figure, name='Others-CCCPS119a')
    [10] Bokeh(figure, name='BHMBCCTHL01')
    [11] Bokeh(figure, name='Others-CCCPS135a')
    [12] Bokeh(figure, name='Others-CCCPS202')
    [13] Bokeh(figure, name='Others-CCCPS98')